In [6]:
from time import time

import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np
import math
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST
from torchvision.datasets import FashionMNIST as FMNIST
from torchvision.datasets import EMNIST
import torchvision.transforms as transforms

import onlinehd

SEED = 1234
device = 'cuda' if torch.cuda.is_available() else 'cpu'
np.random.seed(SEED)
torch.manual_seed(SEED)

In [7]:
# loads simple mnist dataset
def load():
    # fetches data
    x, y = sklearn.datasets.fetch_openml('mnist_784', return_X_y=True)
    x = x.astype(np.float)
    y = y.astype(np.int)
    y = np.array(y)

    # split and normalize
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()

    return x, x_test, y, y_test, scaler

In [8]:
x, x_test, y, y_test, scaler = load()

/var/folders/4_/4812b_4j2_s0smfmxlqdpr600000gn/T/ipykernel_48945/3995914916.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = x.astype(np.float)
/var/folders/4_/4812b_4j2_s0smfmxlqdpr600000gn/T/ipykernel_48945/3995914916.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://nu

In [9]:
x = x.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

scaler = sklearn.preprocessing.Normalizer().fit(x.reshape(-1, 784))
x = torch.tensor(scaler.transform(x.reshape(-1, 784)))
x_test = torch.tensor(scaler.transform(x_test.reshape(-1, 784)))

In [11]:
classes = y.unique().size(0)
features = x.size(1)
model = onlinehd.OnlineHD(classes, features, dim=10000)

if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x_test = x_test.cuda()
    y_test = y_test.cuda()
    model = model.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
model = model.fit(x, y, bootstrap=1.0, lr=0.035, epochs=20)
t = time() - t

Training...
0
10


In [12]:
print('Validating...')
yhat = model(x)
yhat_test = model(x_test)
acc = (y == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{t = :6f}')

Validating...
acc = 0.945410
acc_test = 0.935429
t = 21.452532


In [37]:
import pickle

with open('dnn_adversarial_sample/MNIST_cnn_FGSM.pickle', 'rb') as f:
    mnist_cnn_fgsm = pickle.load(f)
    
with open('dnn_adversarial_sample/MNIST_cnn_DF.pickle', 'rb') as f:
    mnist_cnn_df = pickle.load(f)
    
with open('dnn_adversarial_sample/MNIST_cnn_JSMA.pickle', 'rb') as f:
    mnist_cnn_jsma = pickle.load(f)

In [50]:
mnist_cnn_fgsm_data = torch.from_numpy(scaler.transform(mnist_cnn_fgsm['data']['0.01'].reshape(-1, 784).detach().numpy())).float()
mnist_cnn_fgsm_label = mnist_cnn_fgsm['label'].long()

In [51]:
mnist_cnn_df_data = torch.from_numpy(scaler.transform(mnist_cnn_df['data'])).float()
mnist_cnn_df_label = mnist_cnn_df['label']

In [52]:
mnist_cnn_jsma_data = torch.from_numpy(scaler.transform(mnist_cnn_jsma['data'].reshape(-1, 784))).float()
mnist_cnn_jsma_label = torch.from_numpy(mnist_cnn_jsma['label']).long()

In [54]:
print('Validating...')
yhat = model(x)
yhat_test = model(x_test)
yhat_mnist_fgsm = model(mnist_cnn_fgsm_data)
yhat_mnist_df = model(mnist_cnn_df_data)
yhat_mnist_jsma = model(mnist_cnn_jsma_data)

acc = (y == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
acc_fgsm = (y_test == yhat_mnist_fgsm).float().mean()
acc_df = (y_test == yhat_mnist_df).float().mean()
acc_jsma = (y_test == yhat_mnist_jsma).float().mean()


print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{acc_fgsm = :6f}')
print(f'{acc_df = :6f}')
print(f'{acc_jsma = :6f}')
print(f'{t = :6f}')

Validating...
acc = 0.945410
acc_test = 0.935429
acc_fgsm = 0.932057
acc_df = 0.911257
acc_jsma = 0.898514
t = 21.452532
